# **Machine Learning Language Translation**

[Encoder And Decoder- Neural Machine Learning Language Translation Tutorial With Keras- Deep Learning](https://www.youtube.com/watch?v=f-JCCOHwx1c)

[Keras Blog](https://https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)

We will use a dataset of pairs of English sentences and their French translation, which you can download from [manythings.org/anki](https://manythings.org/anki). The file to download is called fra-eng.zip.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [ ]:
batch_size = 64 # batch size for training
epochs = 100 # Number of epochs to train for
latent_dim = 256 # Latent dimentionality of the encoding space
num_samples = 10000 # Number of samples to train on 
# path to the data txt file on disk
data_path = "/content/fra.txt"

## Vectorize the data

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # we use 'tab' as the "start sequence" character for the targets, and '\n' as "end sequence" character
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
          
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)


In [ ]:
input_texts[50:60], target_texts[50:60]

(['I see.',
  'I try.',
  'I won!',
  'I won!',
  'I won.',
  'Oh no!',
  'Relax.',
  'Relax.',
  'Relax.',
  'Relax.'],
 ['\tAha.\n',
  "\tJ'essaye.\n",
  "\tJ'ai gagné !\n",
  "\tJe l'ai emporté !\n",
  '\tJ’ai gagné.\n',
  '\tOh non !\n',
  '\tCalme-toi.\n',
  '\tDétends-toi\u202f!\n',
  '\tDétendez-vous\u202f!\n',
  '\tRelaxe, Max\u202f!\n'])

In [ ]:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


In [ ]:
print("Number of samples: ", len(input_texts))
print("Number of unique input tokens: ", num_encoder_tokens)
print("Number of unique output tokens: ", num_decoder_tokens)
print("Max sequence length for inputs: ", max_encoder_seq_length)
print("Max sequence length for outputs: ", max_decoder_seq_length)

Number of samples:  10000
Number of unique input tokens:  71
Number of unique output tokens:  92
Max sequence length for inputs:  15
Max sequence length for outputs:  59


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [ ]:
input_token_index

## Turn the sentences into 3 Numpy arrays:



* **encoder_input_data** is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters) containing a one-hot vectorization of the English sentences.
* **decoder_input_data** is a 3D array of shape (num_pairs, max_french_sentence_length, num_french_characters) containg a one-hot vectorization of the French sentences.
* **decoder_target_data** is the same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :].



In [ ]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [ ]:
encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape

((10000, 15, 71), (10000, 59, 92), (10000, 59, 92))

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t+1:, input_token_index[" "]] = 1
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character
            decoder_input_data[i, t-1, target_token_index[char]] = 1
    decoder_input_data[i, t+1:, input_token_index[" "]] = 1
    decoder_target_data[i, t:, target_token_index[" "]] = 1

In [ ]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

## Train a basic LSTM-based Seq2Seq model to predict decoder_target_data given encoder_input_data and decoder_input_data

In [ ]:
latent_dim

256

In [ ]:
# Define an input sequence and prcess it

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# we discard 'encoder_outputs' and only keep the states
encoder_states = [state_h, state_c]

In [ ]:
state_h.shape

TensorShape([None, 256])

In [ ]:
num_decoder_tokens

92

In [ ]:
# set up the decoder using 'encoder_states' as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))

We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the trainnig model, but we will use them in inference.

In [ ]:
  decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

Define the model that will turn: 'encoder_input_data' & 'decoder_input_data' into 'decoder_target_data'

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 58s 437ms/step - loss: 0.1107 - accuracy: 0.6931 - val_loss: 0.0343 - val_accuracy: 0.6638
Epoch 2/100
125/125 [==============================] - 53s 424ms/step - loss: 0.0600 - accuracy: 0.6986 - val_loss: 0.0784 - val_accuracy: 0.6638
Epoch 3/100
125/125 [==============================] - 52s 416ms/step - loss: 0.0576 - accuracy: 0.6986 - val_loss: 0.0428 - val_accuracy: 0.6638
Epoch 4/100
125/125 [==============================] - 52s 416ms/step - loss: 0.0514 - accuracy: 0.6986 - val_loss: 0.0514 - val_accuracy: 0.6638
Epoch 5/100
125/125 [==============================] - 53s 428ms/step - loss: 0.0406 - accuracy: 0.6986 - val_loss: 0.0262 - val_accuracy: 0.6638
Epoch 6/100
125/125 [==============================] - 53s 423ms/step - loss: 0.0325 - accuracy: 0.6986 - val_loss: 0.0179 - val_accuracy: 0.6638
Epoch 7/100
125/125 [==============================] - 54s 431ms/step - loss: 0.0271 - accuracy: 0.6986 - val_loss: 0.0365 -